In [1]:
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as psg
import re
import os

In [2]:
#定义机械压缩去重函数
def yasuo(st):
    for i in range(1,int(len(st)/2)+1):
        for j in range(len(st)):
            if st[j:j+i] == st[j+i:j+2*i]:
                k = j + i
                while st[k:k+i] == st[k+i:k+2*i] and k<len(st):   
                    k = k + i
                st = st[:j] + st[k:]    
    return st
yasuo(st="啊啊啊啊啊啊啊")

'啊'

In [3]:
pattern = re.compile(u'[^\u4e00-\u9fa5a-zA-Z0-9]+')
# 定义函数，用于删除非中文、英文和数字字符
def clean_text(text):
    return re.sub(pattern, '', text)

In [4]:
# 加载停用词表
stopwords = set(pd.read_csv('/home/aistudio/dict/stopwords.txt', header=None, encoding='utf-8', squeeze=True))
# 加载自定义词典
jieba.load_userdict('/home/aistudio/dict/dict.txt')
# 定义需要替换的词和替换成的词
replace_dict = {
    '老师': '教师',
    '医生': '医务工作者',
    # 可以继续添加需要替换的词和替换成的词
}
# 定义选择词性的函数
def select_words(words, pos):
    result = []
    for word, tag in words:
        if tag in pos and word not in stopwords:
            # 替换词
            for k, v in replace_dict.items():
                word = word.replace(k, v)
            result.append(word)
    return ' '.join(result)
# 定义分词函数
def cut_comment(comment, pos):
    # 分词
    words = psg.cut(comment)
    # 选择词性
    result = select_words(words, pos)
    # 返回结果
    return result

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.786 seconds.
Prefix dict has been built successfully.


In [5]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [9]:
# 指定要读取的文件夹路径
folder_path = "data_cut"
folder_path1 = "data_freq"
df1 = pd.read_excel("/home/aistudio/data_天猫.xlsx")
# 对comment列进行操作
df1["comment"] = df1["comment"].apply(yasuo)
df1 = df1.dropna() #删除存在缺失值的行
df1['clean_comment'] = df1['comment'].apply(clean_text)
df1['cut_comment'] = df1['clean_comment'].apply(lambda x: cut_comment(x, ['n', 'a', 'v', 'vn', 'an']))

df2 = pd.read_excel("/home/aistudio/merged.xlsx")
merged_df = df1.append(df2, ignore_index=True)
# 将所有评论合并成一个字符串
comments = merged_df['cut_comment'].str.cat()
# 使用jieba分词
words = jieba.cut(comments)
# 统计词频
word_counts = pd.Series(words).value_counts() 
# 将结果保存到新的Excel文件中
result_df = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
result_df.to_excel("data_freq.xlsx")  
